In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
wiki_movie = pd.read_csv('wiki_movie_plots_deduped.csv')
wiki_plot = wiki_movie.dropna(subset=['Plot'])['Plot']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-1.1-2b-it")

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
model.to(device)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

5 prompts generated by chatgpt4 according to Chronological Overview, Character Analysis, Thematic Exploration, Plot Structure Breakdown, Incorporating Critiques

In [ ]:
prompts = [
    'Outline the main events, focus on character motivations, discuss the climax, and highlight the film’s theme.',
    "Provide a plot overview, detail the main characters' development through challenges and transformations, using key scenes for illustration.",
    'Summarize the plot, delve into the film’s themes, and demonstrate how narrative and style develop these themes with scene examples.',
    'Describe the setup, dissect the plot into rising action, climax, and resolution, and discuss how each segment contributes to the overall impact, including any major twists.',
    'Summarize the plot, integrate various critical perspectives, analyze film style and pacing, and offer a personal interpretation compared to popular reviews.'
]

In [ ]:
import random
random.seed(0)
# This is the prompt format the model expects
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

In [ ]:
original_texts_1000 = wiki_plot.sample(1000)

500

In [ ]:
def preprocess_plot(rewrite_prompt, plot):
    prompt = f'{rewrite_prompt}\n{plot}'
    # USER_CHAT_TEMPLATE.format(prompt=prompt)
    return prompt, USER_CHAT_TEMPLATE.format(prompt=prompt)

In [ ]:
def generate_rewrite(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids.to(device)
    outputs = model.generate(input_ids, max_length=7000)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def postprocess_rewrite(rewrite, input_prompt):
    repete_input = f'user\n{input_prompt}\nmodel\n'
    if str(rewrite).startswith(repete_input):
        return rewrite[len(repete_input):]

In [ ]:
rewrite_data = []
for original_text in original_texts_1000:
    prompt = random.choice(prompts)
    input_prompt, input_text = preprocess_plot(prompt, original_text)
    generated_rewrite = generate_rewrite(input_text)
    rewrite = postprocess_rewrite(generated_rewrite, input_prompt)
    rewrite_data.append({'original_text': original_text, 'rewrite_prompt': prompt, 'rewrite': rewrite})

In [ ]:
rewrite_data = pd.DataFrame(rewrite_data)

In [ ]:
rewrite_data.to_csv('rewrite_data_wikimovie.csv', index = False)